In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import WalmartAmazonStructuredBenchmark

benchmark = WalmartAmazonStructuredBenchmark(data_dir_path="../data/")
benchmark

22:52:53 INFO:Extracting Walmart-Amazon-Structured...
22:52:53 INFO:Reading Walmart-Amazon-Structured record_dict...
22:52:53 INFO:Reading Walmart-Amazon-Structured train.csv...
22:52:53 INFO:Reading Walmart-Amazon-Structured valid.csv...
22:52:53 INFO:Reading Walmart-Amazon-Structured test.csv...


<WalmartAmazonStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Walmart-Amazon/walmart_amazon_exp_data.zip

## Preprocess

In [5]:
field_list = ['title', 'category', 'brand', 'modelno', 'price']

In [6]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record_dict in [benchmark.train_record_dict, benchmark.valid_record_dict, benchmark.test_record_dict]:
    for record in record_dict.values():
        for field in field_list:
            record[field] = clean_str(record[field])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
from entity_embed import PairNumericalizer

pair_numericalizer = PairNumericalizer(field_list)

In [9]:
batch_size = 32
eval_batch_size = 256
datamodule = benchmark.build_matcher_datamodule(
    pair_numericalizer=pair_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

## Training

In [10]:
from entity_embed import Matcher

model = Matcher(
    pair_numericalizer=pair_numericalizer
)

22:52:58 INFO:Load pretrained SentenceTransformer: stsb-distilbert-base
22:52:58 INFO:Did not find folder stsb-distilbert-base
22:52:58 INFO:Search model on server: http://sbert.net/models/stsb-distilbert-base.zip
22:52:58 INFO:Load SentenceTransformer from folder: /home/fjsj/.cache/torch/sentence_transformers/sbert.net_models_stsb-distilbert-base
22:52:58 INFO:Use pytorch device: cuda


In [11]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_f1_at_0.5",
    tb_save_dir='../tb_logs',
    tb_name=f'matcher-{benchmark.dataset_name}'
)

22:52:59 INFO:GPU available: True, used: True
22:52:59 INFO:TPU available: False, using: 0 TPU cores
22:52:59 INFO:Train positive pair count: 576
22:52:59 INFO:Train negative pair count: 5568
22:52:59 INFO:Valid positive pair count: 193
22:52:59 INFO:Valid positive pair count: 1856
22:52:59 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
22:53:02 INFO:
  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 66.4 M
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
66.4 M    Trainable params
0         Non-trainable params
66.4 M    Total params
265.455   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

23:33:36 INFO:Loading the best validation model from ../tb_logs/matcher-Walmart-Amazon-Structured/version_1/checkpoints/epoch=16-step=3263.ckpt...
23:33:36 INFO:Load pretrained SentenceTransformer: stsb-distilbert-base
23:33:36 INFO:Did not find folder stsb-distilbert-base
23:33:36 INFO:Search model on server: http://sbert.net/models/stsb-distilbert-base.zip
23:33:36 INFO:Load SentenceTransformer from folder: /home/fjsj/.cache/torch/sentence_transformers/sbert.net_models_stsb-distilbert-base
23:33:37 INFO:Use pytorch device: cuda


In [12]:
model.validate(datamodule)

{'valid_f1_at_0.3': tensor(0.7778),
 'valid_f1_at_0.5': tensor(0.7844),
 'valid_f1_at_0.7': tensor(0.7725),
 'valid_f1_at_0.9': tensor(0.7486),
 'valid_precision_at_0.3': tensor(0.7586),
 'valid_precision_at_0.5': tensor(0.7865),
 'valid_precision_at_0.7': tensor(0.7892),
 'valid_precision_at_0.9': tensor(0.8121),
 'valid_recall_at_0.3': tensor(0.7979),
 'valid_recall_at_0.5': tensor(0.7824),
 'valid_recall_at_0.7': tensor(0.7565),
 'valid_recall_at_0.9': tensor(0.6943)}

## Testing

In [13]:
model.test(datamodule)

23:34:43 INFO:Test positive pair count: 193
23:34:43 INFO:Test positive pair count: 1856


{'test_f1_at_0.3': tensor(0.7989),
 'test_f1_at_0.5': tensor(0.8022),
 'test_f1_at_0.7': tensor(0.7978),
 'test_f1_at_0.9': tensor(0.7931),
 'test_precision_at_0.3': tensor(0.8162),
 'test_precision_at_0.5': tensor(0.8409),
 'test_precision_at_0.7': tensor(0.8571),
 'test_precision_at_0.9': tensor(0.8903),
 'test_recall_at_0.3': tensor(0.7824),
 'test_recall_at_0.5': tensor(0.7668),
 'test_recall_at_0.7': tensor(0.7461),
 'test_recall_at_0.9': tensor(0.7150)}